In [1]:
#import os
#os._exit(00)

In [ ]:
#!pip uninstall accelerate transformers # https://stackoverflow.com/questions/76781329/importerror-with-transformers-and-accelerate-in-google-colab-seq2seqtrainingarg

In [10]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!pip install accelerate>=0.20.3 transformers
# then restart runtime

In [4]:
!pip install bitsandbytes
!pip install accelerate
!pip install transformers
!pip install langchain
!pip install sentence-transformers
!pip install pypdf
!pip install chromadb
!pip install accelerate

In [5]:
!pip install accelerate
!pip install huggingface_hub
#pip install -i https://pypi.org/simple/ bitsandbytes

In [6]:
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

# Mistral LLM


In [11]:
# if already downloaded:
model_save_name = 'classifier.pt'
path = F"/content/gdrive/MyDrive/AI/models/mistral7B_4bit.pt"
torch.save(model_4bit.state_dict(), path)
# model_4bit = AutoModelForCausalLM.from_pretrained( path, device_map="auto")
# tokenizer = AutoTokenizer.from_pretrained(path)

In [15]:
!ls /content/gdrive/MyDrive/AI/models

mistral7B_4bit.pt


In [7]:
# For the model quantization to be able to run in a free Google Colab notebook
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)
# Version of the Mistral model used
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

# Model and tokenizer
model_4bit = AutoModelForCausalLM.from_pretrained( model_id, device_map="auto",quantization_config=quantization_config, )
tokenizer = AutoTokenizer.from_pretrained(model_id)

# So input sequence in model, embedded in integers, to tokens. tokenizer splits up sentence
# Model is no more than dictionary of tokens to integers. then inverse, integers get redone into tokens

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [17]:
# Way to ask questions to the model:
def askQuestion(prompt, max_new_tokens = 500, temperature = 0.8, top_p = 0.9, do_sample=True, repetition_penalty=1.2):
  """ Encode the input prompt with Mistral instruct format, encode the prompt, use the Quantized Mistral model, decode the reponse and remove the input prompt from this response

      Args:
        prompt (str): Input prompt
        max_new_tokens (int): Specifies the maximum number of new tokens (words or pieces of words) that the model is allowed to generate in response to the prompt.
        temperature (float): Controls the randomness of the model's responses. Lower temperatures (closer to 0) make the model's outputs more deterministic and repetitive,
                             while higher temperatures (closer to 1 or above) increase diversity and creativity in the responses.
        top_p (float): Controls the model's sampling process, where it selects the next token based on a cumulative probability distribution of the possible next tokens.
                       A common setting might be top_p = 0.9, which means the model will only consider the smallest set of tokens whose cumulative probability exceeds 90%
                       for generating the next token. This approach helps to focus the model's choices on more likely outcomes, reducing the
                       chance of selecting very improbable tokens and generally producing more coherent text.

        do_sample (bool): The do_sample parameter controls whether the Large Language Model (LLM) generates text based on a sampling approach rather than simply
                          choosing the most likely next token at each step. When do_sample is set to True, the model uses the probabilities of each token to randomly
                          generate the next token in the sequence. This method introduces variability and creativity in the generated text, making the outputs less
                          deterministic and more diverse.
        repetition_penalty (float): is used to discourage or encourage the repetition of tokens (words or pieces of words) in the text generated by the model.
                                   It modifies the likelihood of tokens based on their previous occurrences in the generated text,
                                   aiming to control redundancy and improve the diversity and quality of the output.

      Returns:
        (str): Clean response to the prompt
  """
  prompt = f"""[INST]{prompt}[/INST]"""
  input = tokenizer.encode(prompt, return_tensors="pt").to("cuda")  # should look online
  output = model_4bit.generate(input, max_new_tokens=max_new_tokens, temperature=temperature, top_p=top_p, repetition_penalty=repetition_penalty, do_sample=do_sample)
  answer = tokenizer.decode(output[0],skip_special_tokens=True).replace(prompt, "" )
  return answer

  # temperature = stochastic influence of generating next word

### Asking Questions

In [ ]:
question = "Enumerate the events between the invasion of ukraine by russia since 2022"
response = askQuestion(question,temperature = 0, top_p = 1.0, do_sample=False, repetition_penalty=0.05) # no creativity, almost totally deterministic
print(response)

In [19]:
# More creative
question = "Enumerate the events between the invasion of ukraine by russia since 2022"
response = askQuestion(question)
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Here is a list of some significant events that have occurred in Ukraine following its invasion by Russia in February 2022:

1. Russian troops invade Ukraine on February 24, 2022.
2. The Ukrainian government declares martial law and mobilizes its military forces to defend against the invasion.
3. Ukraine's president Volodymyr Zelensky addresses the nation and calls for unity and solidarity in the face of the invasion.
4. The European Union (EU) and North Atlantic Treaty Organization (NATO) condemn the invasion and impose economic sanctions on Russia.
5. International organizations such as the United Nations and Amnesty International call for an immediate end to the conflict and violations of human rights.
6. On March 8, 2022, Ukraine announces that it has reclaimed control of the city of Kherson, which had been occupied by Russian troops.
7. On April 9, 2022, Ukraine launches an offensive to retake control of the Donbas region, which has been under Russian-backed separatists' control s